0. Scan glass slides at 20x
1. Create Excel sheet with patient information
2. Screen H&E images to select potentially healthy sections
3. Annotate the Region of Interest (ROI) (Optional)
4. Convert ROI XML to Image (Code: roixml2png.py)
5. Convert H&E images to 1um resolution tif to apply DeepLab (Code: rescale_wsi.py)
6. Annotate 12 tissue compartments, apply DeepLab, and evaluate performance metrics
7. Apply HoVerNet
8. Rotate and crop tissue pieces and DLmask (Code)
9. Rotate HoVerNet segmented nuclei mask (Code)
10. Calculate tissue composition (Code)
11. Calculate epidermis thickness, waviness, roughness
12. Calculate dermis thickness, waviness
13. Calculate ECM anisotropy,Interstitial orientation, Thick gap ratio, Thick gap average
14. Calculate tissue morphology
15. ECM cell count and morphology + std using Manual Threshold method
16. Distance from ECM cell to tissue
17. All celltype morphology, orientation, and distance to tissue using HoVerNet
18. Further add features such as more detailed cell subtyping

In [1]:
import os
import glob
import pandas as pd
import numpy as np
from natsort import natsorted

#utilities
from roixml2png import roixml2png
from rescale_wsi import rescale_wsi
from align_he_dl_cnt import align_he_dl_cnt

In [28]:
NDPI = r'\\fatherserverdw\kyuex\clue images'
oneum = r'\\fatherserverdw\kyuex\clue images\1um'
imcrop = r'\\fatherserverdw\kyuex\clue images\1um\imcrop'
imcroproi = r'\\fatherserverdw\kyuex\clue images\1um\imcrop_roi'

dlmask = r'\\fatherserverdw\kyuex\clue images\1um\classification_v9_combined'
dlcrop = r'\\fatherserverdw\kyuex\clue images\1um\classification_v9_combined\dlcrop'
dlcroproi = r'\\fatherserverdw\kyuex\clue images\1um\classification_v9_combined\dlcrop_roi'

roixml = r'\\fatherserverdw\kyuex\clue images\annotations\roi'
roimask_small = r'\\fatherserverdw\kyuex\clue images\annotations\roi\labeledmask_20rsf'
tissue_area_small = r'\\fatherserverdw\kyuex\clue images\annotations\roi\TA_20rsf'

twelve_annot = r'\\fatherserverdw\kyuex\clue images\annotations\12class'

LUT = r"\\fatherserverdw\kyuex\imlist_all.xlsx"

In [29]:
#Load healthy tissue look up table (LUT)
LUT = pd.read_excel(LUT)
# xmlist = LUT['filename'][(LUT['student score']>1) & (LUT['Block or Slide?']=="Both")]
LUThealthy = LUT[(LUT['student score']>1)]
LUThealthyback = LUThealthy[LUThealthy['body part 1'].str.lower()=="back"]

In [5]:
#Step 4: ROI XML 2 Image
#Saves Image to where XML is in subfolders

xmlist = [os.path.join(roixml,xmlpth.replace('ndpi','xml')) for xmlpth in LUThealthyback['filename']]
[roixml2png(xmlpth,NDPI) for xmlpth in xmlist]
# pd.DataFrame(np.array(roiinfo),columns=['fn','ROIA','TA','ratio']).to_csv(r"\\fatherserverdw\kyuex\ROITA_ratio.csv")

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [6]:
#Step 5: rescale wsi to 1um for applying deeplab
#Saves Image to where wsi is in subfolder
wsis = [os.path.join(NDPI,_) for _ in LUThealthy.filename]
wsis = natsorted(wsis)
for idx,wsi in enumerate(wsis):
    rescale_wsi(wsi,1)
    print(idx,'/',len(wsis))

0 / 216
1 / 216
2 / 216
3 / 216
4 / 216
5 / 216
6 / 216
7 / 216
8 / 216
9 / 216
10 / 216
11 / 216
12 / 216
13 / 216
14 / 216
15 / 216
16 / 216
17 / 216
18 / 216
19 / 216
20 / 216
21 / 216
22 / 216
23 / 216
24 / 216
25 / 216
26 / 216
27 / 216
28 / 216
29 / 216
30 / 216
31 / 216
32 / 216
33 / 216
34 / 216
35 / 216
36 / 216
37 / 216
38 / 216
39 / 216
40 / 216
41 / 216
42 / 216
43 / 216
44 / 216
45 / 216
46 / 216
47 / 216
48 / 216
49 / 216
50 / 216
51 / 216
52 / 216
53 / 216
54 / 216
55 / 216
56 / 216
57 / 216
58 / 216
59 / 216
60 / 216
61 / 216
62 / 216
63 / 216
64 / 216
65 / 216
66 / 216
67 / 216
68 / 216
69 / 216
70 / 216
71 / 216
72 / 216
73 / 216
74 / 216
75 / 216
76 / 216
77 / 216
78 / 216
79 / 216
80 / 216
81 / 216
82 / 216
83 / 216
84 / 216
85 / 216
86 / 216
87 / 216
88 / 216
89 / 216
90 / 216
91 / 216
92 / 216
93 / 216
94 / 216
95 / 216
96 / 216
97 / 216
98 / 216
99 / 216
100 / 216
101 / 216
102 / 216
103 / 216
104 / 216
105 / 216
106 / 216
107 / 216
108 / 216
109 / 216
110 / 216


In [3]:
#8. Rotate and crop tissue pieces and DLmask (Code)
#9. Rotate HoVerNet segmented nuclei mask (Code)

LUTpth = r"\\fatherserverdw\kyuex\datadst\WSI_rotation_crop_LUT.xlsx"
ClUedegrot_df = pd.read_excel(LUTpth).astype("object")

In [30]:
cntsrc = r'\\fatherserverdw\kyuex\clue images\hovernet_out\json'
def hasjson(row):
    fn,ext = os.path.splitext(row)
    return os.path.exists(os.path.join(cntsrc,fn+'.json'))
LUThealthyback['hasjson']=LUThealthyback['filename'].apply(lambda row: hasjson(row))

C:\Users\pwu\AppData\Local\Temp\ipykernel_7376\2748685258.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LUThealthyback['hasjson']=LUThealthyback['filename'].apply(lambda row: hasjson(row))


In [32]:
dlsrc = r'\\fatherserverdw\kyuex\clue images\1um\classification_v9_combined'
def hasdl(row):
    fn,ext = os.path.splitext(row)
    return os.path.exists(os.path.join(dlsrc,fn+'.tif'))
LUThealthyback['hasdl']=LUThealthyback['filename'].apply(lambda row: hasdl(row))

C:\Users\pwu\AppData\Local\Temp\ipykernel_7376\55481327.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LUThealthyback['hasdl']=LUThealthyback['filename'].apply(lambda row: hasdl(row))


In [35]:
roisrc = r'\\fatherserverdw\kyuex\clue images\annotations\roi\labeledmask_20rsf'
def hasroi(row):
    fn,ext = os.path.splitext(row)
    return os.path.exists(os.path.join(roisrc,fn+'.png'))
LUThealthyback['hasroi']=LUThealthyback['filename'].apply(lambda row: hasroi(row))

C:\Users\pwu\AppData\Local\Temp\ipykernel_7376\556606825.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LUThealthyback['hasroi']=LUThealthyback['filename'].apply(lambda row: hasroi(row))


In [36]:
LUThealthyback

,index,gender,age,pid,path,filename,year,sample id,label name,sub label,...,score note,student score,ROI ANNOTATED (0 :not annotated 1.annotated),roi reviewed,Nroi,Inbal Score,Inbal comment,hasjson,hasdl,hasroi
52,1141,F,85.604454,471,\\fatherserverdw\Q\research\images\clue_batch2...,2022-06-07 14.27.44.ndpi,10,3702,10-3702,1,...,NaN,5,1,1.0,NaN,NaN,NaN,True,True,True
53,1142,F,85.604454,471,\\fatherserverdw\Q\research\images\clue_batch2...,2022-06-07 14.32.24.ndpi,10,3702,10-3702,2,...,NaN,5,1,1.0,NaN,NaN,NaN,True,True,True
54,1143,F,85.604454,471,\\fatherserverdw\Q\research\images\clue_batch2...,2022-06-07 14.34.59.ndpi,10,3702,10-3702,3,...,NaN,5,1,1.0,NaN,NaN,NaN,True,True,True
55,1144,F,85.604454,471,\\fatherserverdw\Q\research\images\clue_batch2...,2022-06-07 14.37.19.ndpi,10,3702,10-3702,4,...,middle of tissue is physically damaged,3,1,1.0,NaN,NaN,NaN,True,True,True
105,1173,M,92.435616,484,\\fatherserverdw\Q\research\images\clue_batch2...,2022-06-07 17.24.15.ndpi,10,6634,10-6634,7,...,sun damage,2,1,1.0,NaN,NaN,NaN,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
967,671,F,72.712329,301,\\fatherserverdw\kyuex\clue images\2022-06-28 ...,2022-06-28 15.22.44.ndpi,7,6405,07-6405,A4,...,NaN,2,1,1.0,NaN,NaN,NaN,True,True,True
991,1001,M,63.372603,418,\\fatherserverdw\kyuex\clue images\2022-06-28 ...,2022-06-28 23.56.18.ndpi,9,5630,09-5630,B,...,NaN,5,1,1.0,NaN,NaN,NaN,False,True,True
1140,390,M,83.120548,177,\\fatherserverdw\kyuex\clue images\2022-07-07 ...,2022-07-07 17.52.32.ndpi,5,9840,05-9840,2,...,it’s not focused,3,1,1.0,NaN,3.0,"there is residual bcc, even in the green box, ...",True,True,True
1141,274,F,69.970656,126,\\fatherserverdw\kyuex\clue images\2022-07-07 ...,2022-07-07 18.26.18.ndpi,4,13043,04-13043,B,...,"some sun damage, collagen damage",3,1,1.0,NaN,3.0,"middle piece is okay, others have too much sca...",True,True,True


In [38]:
LUThealthyback2 = LUThealthyback[(LUThealthyback['hasjson']==True) & (LUThealthyback['hasdl']==True) & (LUThealthyback['hasroi']==True)]
len(LUThealthyback2),len(LUThealthyback[(LUThealthyback['hasjson']==True)]),len(LUThealthyback[(LUThealthyback['hasdl']==True)]),len(LUThealthyback)

(53, 53, 66, 66)

In [23]:
LUThealthyback2 = LUThealthyback2[54:]

In [39]:
len(LUThealthyback2)

53

In [46]:
LUThealthyback2=LUThealthyback2.reset_index()

In [47]:
cntsrc = r'\\fatherserverdw\kyuex\clue images\hovernet_out\json'
roisrc = r'\\fatherserverdw\kyuex\clue images\annotations\roi\labeledmask_20rsf'
wsisrc = r'\\fatherserverdw\kyuex\clue images'
dlsrc = r'\\fatherserverdw\kyuex\clue images\1um\classification_v9_combined'
dst = r'\\fatherserverdw\kyuex\datadst\20220929'

for idx,row in LUThealthyback2.iterrows():
    print(idx,'/',len(LUThealthyback2))
    fn,ext = os.path.splitext(row['filename'])
    rotationdf = align_he_dl_cnt(dst,fn,wsisrc,dlsrc,cntsrc,roisrc)

0 / 53
section N:  1 / 1


error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\imgwarp.cpp:1724: error: (-215:Assertion failed) dst.cols < SHRT_MAX && dst.rows < SHRT_MAX && src.cols < SHRT_MAX && src.rows < SHRT_MAX in function 'cv::remap'


In [ ]:
# rotationdf.to_csv('54toend.csv')